In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#matplot lib, to see the images
%matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt

#OpenCv is the main librarie of image manipulation
!pip install opencv-python
import cv2
import numpy as np
import dlib

#to visualize the images with better quality
import os
from IPython.display import Image

#to import from website
!pip install wget
import wget

print('ready!')

In [ ]:
!git clone  https://github.com/italojs/facial-landmarks-recognition

In [ ]:
### this is the path of image
url='https://files.alerta.rcnradio.com/alerta_bogota/public/styles/article_desktop/public/2019-10/el_man_0.jpg?itok=wWjcWmPw'
wget.download(url, 'sa.jpg')
url2='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Scarlett_Johansson_in_Kuwait_01b-tweaked.jpg/220px-Scarlett_Johansson_in_Kuwait_01b-tweaked.jpg'
wget.download(url2, 'sj.jpg')
print('Images Downloaded!')
url3 = "https://www.cnet.com/a/img/K9M4-6WeU10J3QC-AXwJSDxhd8M=/1200x675/2020/09/02/2b8810d6-ab1a-4bd1-8f0f-1f05cc9099c7/bond-poster.jpg"
wget.download(url1, 'jb.jpg')
#jinny BP
# url4 = "https://www.brighttv.co.th/wp-content/uploads/2021/07/89f81a8c5ca14803a73345e8397c21b7.jpeg"
# wget.download(url4, 'jbp.jpg')

In [ ]:
pattern='./facial-landmarks-recognition/shape_predictor_68_face_landmarks.dat'

In [ ]:
#IMAGE 1  : principal face
img1 = cv2.imread("./jbp.jpg") 
Image("./jbp.jpg")

In [ ]:
#IMAGE 2  :  Image2-background body

img2 = cv2.imread("./sa.jpg") 
Image("./sa.jpg")

In [ ]:
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(img1_gray)
plt.imshow(img1_gray, cmap='gray')
plt.show 

In [ ]:
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
plt.imshow(img2_gray, cmap='gray')
plt.show


Detecting face landmark

Here, we detect the facial pattern in the images, such as the face shape as a landmark reference


In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(pattern)
faces = detector(img1_gray)
for face in faces:
    landmarks = predictor(img1_gray, face)
    landmarks_points = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points.append((x, y))

print('next')



Now we calculate the convex hull


In [ ]:
points = np.array(landmarks_points, np.int32)
convexhull = cv2.convexHull(points)
print('next')

Detecting face landmark in image 2

In [ ]:
faces2 = detector(img2_gray)
for face in faces2:
  landmarks = predictor(img2_gray, face)
  landmarks_points2 = []
  for n in range(0, 68):
    x = landmarks.part(n).x
    y = landmarks.part(n).y
    landmarks_points2.append((x, y))
    #cv2.circle(img2, (x, y), 3, (0, 255, 0), -1) #This line help to print landmarks in the face
    
print('next')

and convex hull to image 2

In [ ]:
points2 = np.array(landmarks_points2, np.int32)
convexhull2 = cv2.convexHull(points2)
print('next')


Splitting the face into triangles

We need to divide both faces into triangles, to match the shape of both faces
Applying Delaunay triangulation to image 1

"For morphing images the Delaunay triangulation provides a 'good' way to create a triangular mesh from points that are going to be moved. The triangular shapes are distorted from one image to the next " from Delaunay triangulation


In [ ]:
rect = cv2.boundingRect(convexhull)#find the rectangle sourrounding convex hull
subdiv = cv2.Subdiv2D(rect)
subdiv.insert(landmarks_points)
triangles = subdiv.getTriangleList()
triangles = np.array(triangles, dtype=np.int32)
for t in triangles:
   pt1 = (t[0], t[1])
   pt2 = (t[2], t[3])
   pt3 = (t[4], t[5])
   #cv2.line(img, pt1, pt2, (0, 0, 255), 2) #this unselected lines help o vizualize the triangles 
   #cv2.line(img, pt2, pt3, (0, 0, 255), 2) #on the face
   #cv2.line(img, pt3, pt1, (0, 0, 255), 2)

print('next')



we need the next black pattern to put the face from image 1


In [ ]:
img2_new_face = np.zeros_like(img2)
plt.imshow(img2_new_face)
plt.show



Next is a function to extract the indexes of the triangles from the landmarks point array:


In [ ]:
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index
print('next')


Finding the indexes of each triangle (or specific landmarks points each triangle connects).


In [ ]:
indexes_triangles = []
for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)
        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)
        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)
        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)
        #cv2.line(img, pt1, pt2, (0, 0, 255), 2)
        #cv2.line(img, pt2, pt3, (0, 0, 255), 2)
        #cv2.line(img, pt1, pt3, (0, 0, 255), 2)
print('next')


Triangulation of the 2nd face from the 1st face using Delaunay Triangulation


In [ ]:
# Triangulation of both faces
for triangle_index in indexes_triangles:
    # Triangulation of the first face
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)

    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1
    cropped_triangle = img1[y: y + h, x: x + w]
    cropped_tr1_mask = np.zeros((h, w), np.uint8)
    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                      [tr1_pt2[0] - x, tr1_pt2[1] - y],
                      [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr1_mask, points, 255)
    cropped_triangle = cv2.bitwise_and(cropped_triangle, cropped_triangle,
                                       mask=cropped_tr1_mask)
    #cv2.line(img, tr1_pt1, tr1_pt2, (0, 0, 255), 2)
    #cv2.line(img, tr1_pt3, tr1_pt2, (0, 0, 255), 2)
    #cv2.line(img, tr1_pt1, tr1_pt3, (0, 0, 255), 2)

     # Triangulation of second face
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2
    cropped_triangle2 = img2[y: y + h, x: x + w]
    cropped_tr2_mask = np.zeros((h, w), np.uint8)
    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                       [tr2_pt2[0] - x, tr2_pt2[1] - y],
                       [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)
    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)
    cropped_triangle2 = cv2.bitwise_and(cropped_triangle2, cropped_triangle2,
                                       mask=cropped_tr2_mask)
    #cv2.line(img2, tr2_pt1, tr2_pt2, (0, 0, 255), 2)
    #cv2.line(img2, tr2_pt3, tr2_pt2, (0, 0, 255), 2)
    #cv2.line(img2, tr2_pt1, tr2_pt3, (0, 0, 255), 2)
    
    # Let's Warp triangles
    points = np.float32(points)
    points2 = np.float32(points2)
    M = cv2.getAffineTransform(points, points2)
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)##

    # *****Reconstructing destination face************************************************
    img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
    img2_new_face_rect_area_gray= cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)

    #To remove the lines between triangles
    _, mask_triangles_designed=cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle=cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

    img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
    img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area

    # Face swapped (putting 1st face into 2nd face)
    img2_face_mask = np.zeros_like(img2_gray)
    img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
    img2_face_mask = cv2.bitwise_not(img2_head_mask)

    img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
    result = cv2.add(img2_head_noface, img2_new_face)

    #plt.imshow(img2_new_face)
    #plt.imshow(result)
    plt.imshow(result[:,:,::-1])

In [ ]:
!pwd

In [ ]:
plt.imsave('/kaggle/working/save_image.jpg', result[:,:,::-1])